In [18]:
import os
import torch
torch.cuda.empty_cache()
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision 
import torchvision.transforms as transforms
import scipy.io as sio

In [19]:
resnet50_feature = np.empty((0, 2048))
resnet50_label = np.empty((1,0))

In [20]:
transform_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(100),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

PREPARING THE DATA

In [21]:
from zipfile import ZipFile

file_name = "tiny-UCF101.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()

In [ ]:
import builtins as bins

path_to_dataset = 'tiny-UCF101'
categories = sorted(os.listdir(path_to_dataset))

l = list(range(0, 102))
labelDict = dict(bins.zip(categories, l))
labelDict

Custom Dataset

In [23]:
imgPaths = []
labels = []
directory1 = 'tiny-UCF101'
 
for directory2 in sorted(os.listdir(directory1)):
  # print(directory2)
  for fileName in sorted(os.listdir(os.path.join(directory1, directory2))):
    f = os.path.join(directory1, directory2, fileName)
    imgPaths.append(f)
    labels.append(labelDict[directory2])
    # print (f , '-', labelDict[directory2])
    # # checking if it is a file
    # if os.path.isfile(f):
    #     print(f)

print(imgPaths[5])
print(labels[5])

tiny-UCF101/ApplyEyeMakeup/00006.jpg
0


In [24]:
import cv2
from torch.utils.data import Dataset, DataLoader

class UCF101Dataset(Dataset):
  def __init__(self, image_labels, image_paths, transform = False):
    self.image_labels = image_labels
    self.image_paths = image_paths
    self.transform = transform
  
  def __len__(self):
    return len(self.image_paths)

  def __getitem__(self, idx):
    file_path = self.image_paths[idx]
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    label = self.image_labels [idx] 

    img = self.transform(img)

    return img, label

dataset = UCF101Dataset(labels, imgPaths, transform_test)




In [25]:
#check
print(dataset[144][0].shape)
print(dataset[144][1])

torch.Size([3, 100, 100])
0


DATALOADER

In [26]:
dataLoader = DataLoader(dataset, batch_size=15, shuffle=True)
#check
print(next(iter(dataLoader))[0].shape)
print(next(iter(dataLoader))[1].shape)

torch.Size([15, 3, 100, 100])
torch.Size([15])


LOADING THE MODEL

In [27]:
#import resnet
resnet50 = torchvision.models.resnet50(pretrained=True)
resnet50

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [28]:
# # the out feature in this 1000 - but we need 101
# # so changing the claifier head

# #freezing the parameters of other layers
# for p in resnet50.parameters():
#     p.requires_grad = False

# cnn_feat_dim = resnet50.fc.in_features 
# num_classes = 101

resnet50.fc = nn.Sequential(
    nn.Linear(resnet50.fc.in_features ,2048),
    nn.ReLU(),
    nn.Linear(2048, 2048)
)

# # Allow parameter changes in the fc layer
# for p in resnet50.fc.parameters():
#     p.requires_grad = True

EXTRACTING FEATURES



In [29]:
device = torch.device("cuda:0")
resnet50.to(device)

# # using lambda to filter out only the parameters that we have to change
# params_non_frozen = filter(lambda p:p.requires_grad==True, resnet50.parameters())

# opt = optim.SGD(params_non_frozen, lr=0.01, momentum=0.9)
# loss_fn = nn.CrossEntropyLoss()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [38]:

import numpy as np
resnet50_feature = np.empty((0, 2048))
resnet50_label = np.empty((1,0))
print("the shape is ",resnet50_feature.shape)
count = 0
for idx, data in enumerate(dataLoader):
    images, labels = data
    # print(labels.shape)
    images = images.to(device)

    with torch.no_grad():
      resnet50FeatureBatch = resnet50(images)
      count = count + resnet50FeatureBatch.shape[0]

    resnet50_feature = np.append(resnet50_feature, resnet50FeatureBatch.detach().cpu().numpy(), axis =0)
    print(np.array(resnet50_feature).shape)
    # resnet50_feature.append(resnet50FeatureBatch.cpu().detach().numpy())
    # resnet50_label.append(labels.cpu())
    resnet50_label = np.append(resnet50_label, labels.detach().cpu().numpy())

print(np.array(resnet50_feature).shape)
print(np.array(resnet50_label).shape)
print(count)
sio.savemat('resnet50.mat', mdict={'feature': resnet50_feature, 'label': resnet50_label})


the shape is  (0, 2048)
(15, 2048)
(30, 2048)
(45, 2048)
(60, 2048)
(75, 2048)
(90, 2048)
(105, 2048)
(120, 2048)
(135, 2048)
(150, 2048)
(165, 2048)
(180, 2048)
(195, 2048)
(210, 2048)
(225, 2048)
(240, 2048)
(255, 2048)
(270, 2048)
(285, 2048)
(300, 2048)
(315, 2048)
(330, 2048)
(345, 2048)
(360, 2048)
(375, 2048)
(390, 2048)
(405, 2048)
(420, 2048)
(435, 2048)
(450, 2048)
(465, 2048)
(480, 2048)
(495, 2048)
(510, 2048)
(525, 2048)
(540, 2048)
(555, 2048)
(570, 2048)
(585, 2048)
(600, 2048)
(615, 2048)
(630, 2048)
(645, 2048)
(660, 2048)
(675, 2048)
(690, 2048)
(705, 2048)
(720, 2048)
(735, 2048)
(750, 2048)
(765, 2048)
(780, 2048)
(795, 2048)
(810, 2048)
(825, 2048)
(840, 2048)
(855, 2048)
(870, 2048)
(885, 2048)
(900, 2048)
(915, 2048)
(930, 2048)
(945, 2048)
(960, 2048)
(975, 2048)
(990, 2048)
(1005, 2048)
(1020, 2048)
(1035, 2048)
(1050, 2048)
(1065, 2048)
(1080, 2048)
(1095, 2048)
(1110, 2048)
(1125, 2048)
(1140, 2048)
(1155, 2048)
(1170, 2048)
(1185, 2048)
(1200, 2048)
(1215, 2

In [40]:
import scipy
mat = scipy.io.loadmat('resnet50.mat')

data = mat['feature'] # use the key for data here
target = mat['label'] # use the key for target here

data = torch.from_numpy(data).float()
target = torch.from_numpy(target).int() # change type to your use case

print(data.shape)
print(target.shape)

torch.Size([13320, 2048])
torch.Size([1, 13320])
